In [1]:
!pip install scikit-posthocs
import cv2
import os
from skimage.feature import hog
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp
import scipy.stats as stats


In [ ]:
# Caminho do diretório
dir_path = "/content/basset_hound"
dir_path2 = "/content/york_terrier"
dir_path3 = "/content/abyssinian"
dir_path4 = "/content/egyptian"

# Cria o diretório
os.makedirs(dir_path, exist_ok=True)
os.makedirs(dir_path2, exist_ok=True)
os.makedirs(dir_path3, exist_ok=True)
os.makedirs(dir_path4, exist_ok=True)


In [ ]:
# Lista de diretórios
diretorios = ['/content/basset_hound', '/content/york_terrier', '/content/abyssinian', '/content/egyptian']

# Criar DataFrames vazios
df1 = pd.DataFrame()
df2 = pd.DataFrame()

# Para cada diretório na lista de diretórios
for diretorio in diretorios:
    # Para cada arquivo no diretório
    for nome_arquivo in os.listdir(diretorio):
        if nome_arquivo.endswith(('.png','.jpg', '.jpeg')):
            caminho_arquivo = os.path.join(diretorio, nome_arquivo)
            imagem = cv2.imread(caminho_arquivo)
            if imagem is not None:
                # Redimensionar a imagem para 128x128
                imagem_redimensionada = cv2.resize(imagem, (128, 128))

                # Aplicar HOG com pixels_per_cell=(16,16)
                fd1_temp, hog_image1 = hog(imagem_redimensionada, orientations=9, pixels_per_cell=(16,16),
                                       cells_per_block=(2, 2), visualize=True, multichannel=True)

                # Aplicar HOG com pixels_per_cell=(20,20)
                fd2_temp, hog_image2 = hog(imagem_redimensionada, orientations=9, pixels_per_cell=(20,20),
                                       cells_per_block=(2, 2), visualize=True, multichannel=True)

                # Adicionar os descritores HOG a um DataFrame pandas temporário
                df1_temp = pd.DataFrame(fd1_temp).transpose()
                df2_temp = pd.DataFrame(fd2_temp).transpose()

                # Adicionar uma coluna 'label' para indicar se a imagem é de um cachorro ou gato
                df1_temp['label'] = 'cachorro' if 'basset_hound' in diretorio or 'york_terrier' in diretorio else 'gato'
                df2_temp['label'] = 'cachorro' if 'basset_hound' in diretorio or 'york_terrier' in diretorio else 'gato'

                # Anexar os DataFrames temporários aos DataFrames principais
                df1 = df1.append(df1_temp)
                df2 = df2.append(df2_temp)
            else:
                print(f"Não foi possível abrir a imagem: {caminho_arquivo}")
        else:
            print(f"Arquivo ignorado (não é uma imagem): {caminho_arquivo}")

# Salvar os DataFrames como CSV
df1.to_csv('dataset1.csv', index=False)
df2.to_csv('dataset2.csv', index=False)


In [ ]:
# Carregar o primeiro conjunto de dados
df1 = pd.read_csv('dataset1.csv')

# Separar os atributos das classes
y = df1['label']
X = df1.drop('label', axis=1)

# Aplicar PCA com diferentes percentuais de variância explicada
percentuais = [0.75, 0.85]
for percentual in percentuais:
    # Inicializar o objeto PCA com o percentual desejado
    pca = PCA(n_components=percentual)

    # Aplicar o PCA aos dados
    X_reduced = pca.fit_transform(X)

    # Criar um novo DataFrame com os atributos reduzidos
    df_reduced = pd.DataFrame(X_reduced)
    # Adicionar a coluna 'label' de volta ao DataFrame
    df_reduced['label'] = y

    # Salvar o DataFrame como CSV
    df_reduced.to_csv(f'dataset1_pca_{percentual}.csv', index=False)

In [ ]:
# modelo de CNN pré-treinado

modelo = VGG16(weights='imagenet', include_top=False, pooling='avg')

recursos_vgg_avg = []
recursos_vgg_max = []
rotulos = []

# Para cada diretório na lista de diretórios
for diretorio in diretorios:
    # Para cada arquivo no diretório
    for nome_arquivo in os.listdir(diretorio):
        if nome_arquivo.endswith(('.png','.jpg', '.jpeg')):
            caminho_arquivo = os.path.join(diretorio, nome_arquivo)
            imagem = cv2.imread(caminho_arquivo)
            if imagem is not None:
                # Redimensionar a imagem para 224x224
                imagem_redimensionada = cv2.resize(imagem, (224, 224))

                # Converter a imagem para o formato esperado pelo VGG16
                imagem_vgg = np.expand_dims(imagem_redimensionada, axis=0)
                imagem_vgg = preprocess_input(imagem_vgg)

                # Usar o VGG16 para extrair recursos
                recursos_avg = modelo.predict(imagem_vgg)

                # Mudar pooling para 'max' e extrair recursos novamente
                modelo.pooling = 'max'
                recursos_max = modelo.predict(imagem_vgg)

                # Adicionar os recursos à lista
                recursos_vgg_avg.append(recursos_avg.flatten())
                recursos_vgg_max.append(recursos_max.flatten())

                # Adicionar o rótulo à lista de rótulos
                rotulo = 'cachorro' if 'basset_hound' in diretorio or 'york_terrier' in diretorio else 'gato'
                rotulos.append(rotulo)

# Criar DataFrames com os recursos extraídos e os rótulos correspondentes
df_vgg_avg = pd.DataFrame(recursos_vgg_avg)
df_vgg_max = pd.DataFrame(recursos_vgg_max)
df_vgg_avg['label'] = rotulos
df_vgg_max['label'] = rotulos

df_vgg_avg.to_csv('dataset_vgg_avg.csv', index=False)
df_vgg_max.to_csv('dataset_vgg_max.csv', index=False)

In [ ]:

df1 = pd.read_csv('dataset1.csv')
df2 = pd.read_csv('dataset2.csv')
df_vgg_avg = pd.read_csv('dataset_vgg_avg.csv')
df_vgg_max = pd.read_csv('dataset_vgg_max.csv')
dataset1_pca = pd.read_csv('dataset1_pca_0.85.csv')
dataset2_pca = pd.read_csv('dataset1_pca_0.75.csv')
datasets = [df1, df2, df_vgg_avg, df_vgg_max, dataset1_pca, dataset2_pca]


resultados = []

# Para cada dataset
for i, df in enumerate(datasets):
    # Separar os atributos das classes
    X = df.drop('label', axis=1)
    y = df['label']

    # Para cada valor de k de 1 a 10
    for k in range(1, 11):

        knn = KNeighborsClassifier(n_neighbors=k)

        # 10-fold cross-validation
        scores = cross_val_score(knn, X, y, cv=StratifiedKFold(10))
        resultados.append(('Dataset'+str(i+1), '10-fold CV', k, scores.mean(), scores.std()))

        # Treinos com a proporção 90/10, 80/20 e 70/30
        for test_size in [0.1, 0.2, 0.3]:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            resultados.append(('Dataset'+str(i+1), str(int(test_size*100))+'%', k, acc))

print(resultados)
# Converter os resultados em um DataFrame pandas para facilitar a visualização
resultados_df = pd.DataFrame(resultados, columns=['Dataset', 'Método', 'k', 'Acurácia', 'Desvio padrão'])

# Salvar o DataFrame como CSV
resultados_df.to_csv('resultados.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# Lendo o arquivo CSV
df = pd.read_csv('resultados.csv')

# Criando uma figura e um conjunto de subtramas
fig, ax =plt.subplots(1,1)

# Escondendo os eixos
ax.axis('tight')
ax.axis('off')

# Criando a tabela
ax.table(cellText=df.values,
         colLabels=df.columns,
         cellLoc = 'center',
         loc='center')

# Salvando a figura
plt.savefig('tabela.png')

In [ ]:
## Prática DT e NB
df1 = pd.read_csv('dataset1.csv')
df2 = pd.read_csv('dataset2.csv')
df_vgg_avg = pd.read_csv('dataset_vgg_avg.csv')
df_vgg_max = pd.read_csv('dataset_vgg_max.csv')
dataset1_pca = pd.read_csv('dataset1_pca_0.85.csv')
dataset2_pca = pd.read_csv('dataset1_pca_0.75.csv')
datasets = [df1, df2, df_vgg_avg, df_vgg_max, dataset1_pca, dataset2_pca]

# Lista de tuplas (nome, DataFrame)
datasets = [('df1', df1), ('df2', df2), ('df_vgg_avg', df_vgg_avg), ('df_vgg_max', df_vgg_max), ('dataset1_pca', dataset1_pca), ('dataset2_pca', dataset2_pca)]

# Lista de divisões de treinamento/teste
splits = [(0.7, 0.3), (0.8, 0.2), (0.9, 0.1)]

# Inicializar um DataFrame para armazenar os resultados
results = pd.DataFrame(columns=['Base', 'Treinamento/Teste', 'md = 3', 'md = 4', 'md = 5', 'md = 6', 'md = 7', 'md = 8', 'md = 9', 'md = 10'])

# Loop sobre os conjuntos de dados
for name, dataset in datasets:
    # Separar as características e o alvo
    X = dataset.drop('label', axis=1)
    y = dataset['label']

    # Loop sobre as divisões de treinamento/teste
    for train_size, test_size in splits:
        # Dividir o conjunto de dados em treino e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

        # Inicializar uma linha para este conjunto de dados e divisão
        row = {'Base': name, 'Treinamento/Teste': f'{int(train_size * 100)}/{int(test_size * 100)}'}

        # Loop sobre os valores de max_depth
        for max_depth in range(3, 11):
            # Inicializar a Árvore de Decisão
            clf = DecisionTreeClassifier(max_depth=max_depth)

            # Treinar o modelo e calcular a acurácia no conjunto de teste
            clf.fit(X_train, y_train)
            accuracy = clf.score(X_test, y_test)

            # Adicionar a acurácia à linha
            row[f'md = {max_depth}'] = accuracy

        # Adicionar a linha ao DataFrame de resultados
        results = results.append(row, ignore_index=True)

    # Calcular a acurácia usando validação cruzada 10-fold para cada max_depth
    row_cv = {'Base': name, 'Treinamento/Teste': '10-fold CV'}
    for max_depth in range(3, 11):
        clf_cv = DecisionTreeClassifier(max_depth=max_depth)
        scores_cv = cross_val_score(clf_cv, X, y, cv=10)
        row_cv[f'md = {max_depth}'] = scores_cv.mean()
    results = results.append(row_cv, ignore_index=True)

# Imprimir o DataFrame de resultados
print(results)


results.to_excel('results.xlsx')
results.to_csv('results.csv')


In [ ]:
# Inicializar um DataFrame para armazenar os resultados
results_nb = pd.DataFrame(columns=['Base', 'Treinamento/Teste', 'GaussianNB'])

# Loop sobre os conjuntos de dados
for name, dataset in datasets:
    # Separar as características e o alvo
    X = dataset.drop('label', axis=1)
    y = dataset['label']

    # Loop sobre as divisões de treinamento/teste
    for train_size, test_size in splits:
        # Dividir o conjunto de dados em treino e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

        # Inicializar uma linha para este conjunto de dados e divisão
        row = {'Base': name, 'Treinamento/Teste': f'{int(train_size * 100)}/{int(test_size * 100)}'}

        # Inicializar o Gaussian Naive Bayes
        gnb = GaussianNB()

        # Treinar o modelo e calcular a acurácia no conjunto de teste
        gnb.fit(X_train, y_train)
        accuracy = gnb.score(X_test, y_test)

        # Adicionar a acurácia à linha
        row['GaussianNB'] = accuracy

        # Adicionar a linha ao DataFrame de resultados
        results_nb = results_nb.append(row, ignore_index=True)

    # Calcular a acurácia usando validação cruzada 10-fold para GaussianNB
    row_cv = {'Base': name, 'Treinamento/Teste': '10-fold CV'}
    gnb_cv = GaussianNB()
    scores_cv = cross_val_score(gnb_cv, X, y, cv=10)
    row_cv['GaussianNB'] = scores_cv.mean()
    results_nb = results_nb.append(row_cv, ignore_index=True)

# Imprimir o DataFrame de resultados
print(results_nb)
results_nb.to_excel('results.xlsx')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score

# Carregar os conjuntos de dados
df1 = pd.read_csv('dataset1.csv')
df2 = pd.read_csv('dataset2.csv')
df_vgg_avg = pd.read_csv('dataset_vgg_avg.csv')
df_vgg_max = pd.read_csv('dataset_vgg_max.csv')
dataset1_pca = pd.read_csv('dataset1_pca_0.85.csv')
dataset2_pca = pd.read_csv('dataset1_pca_0.75.csv')

# Função para realizar experimentos com MLP
def experiment_mlp(X, y, num_neurons, activation, max_iter, learning_rate):
    # Dividir os dados em treinamento e teste (70/30, 80/20, 90/10)
    acc_scores = {'70/30': [], '80/20': [], '90/10': [], '10-fold CV': []}

    for test_size in [0.3, 0.2, 0.1]:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

        # Inicializar o classificador MLP
        mlp = MLPClassifier(hidden_layer_sizes=num_neurons, activation=activation, max_iter=max_iter,
                            learning_rate_init=learning_rate, random_state=42)

        # Treinar o modelo
        mlp.fit(X_train, y_train)

        # Avaliar a acurácia nos dados de teste
        y_pred = mlp.predict(X_test)
        acc_test = accuracy_score(y_test, y_pred)
        acc_scores[f'{int((1-test_size)*100)}/{int(test_size*100)}'].append(acc_test)

        # Realizar 10-fold cross-validation
        acc_cv = cross_val_score(mlp, X, y, cv=10)
        acc_scores['10-fold CV'].append(np.mean(acc_cv))

    return acc_scores

# Experimento 1: Fixar número de neurônios e variar funções de ativação
num_neurons = int((X.shape[1] + 2) / 2)  # A = (número de atributos + número de classes) / 2
activation_functions = ['identity', 'logistic', 'tanh', 'relu']

for base, X, y in [('df1', df1.iloc[:, :-1], df1.iloc[:, -1]),
                  ('df2', df2.iloc[:, :-1], df2.iloc[:, -1]),
                  ('df_vgg_avg', df_vgg_avg.iloc[:, :-1], df_vgg_avg.iloc[:, -1]),
                  ('df_vgg_max', df_vgg_max.iloc[:, :-1], df_vgg_max.iloc[:, -1]),
                  ('dataset1_pca', dataset1_pca.iloc[:, :-1], dataset1_pca.iloc[:, -1]),
                  ('dataset2_pca', dataset2_pca.iloc[:, :-1], dataset2_pca.iloc[:, -1])]:
    plt.figure(figsize=(12, 6))
    for activation in activation_functions:
        acc_scores = experiment_mlp(X, y, num_neurons, activation, 1000, 0.001)
        plt.plot(acc_scores['70/30'], label=f'{activation} (70/30)')
        plt.plot(acc_scores['80/20'], label=f'{activation} (80/20)')
        plt.plot(acc_scores['90/10'], label=f'{activation} (90/10)')
        plt.plot(acc_scores['10-fold CV'], label=f'{activation} (10-fold CV)')

    plt.title(f'Experimento 1 - Base: {base}')
    plt.xlabel('Experimento')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.show()


FileNotFoundError: ignored

In [ ]:
# Função para realizar experimentos com MLP (Passo 2)
def experiment_mlp_neurons(X, y, num_neurons_list, activation, max_iter, learning_rate):
    acc_scores = {'Num Neurons': [], 'Accuracy': []}

    for num_neurons in num_neurons_list:
        for test_size in [0.3, 0.2, 0.1]:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

            mlp = MLPClassifier(hidden_layer_sizes=num_neurons, activation=activation, max_iter=max_iter,
                                learning_rate_init=learning_rate, random_state=42)

            mlp.fit(X_train, y_train)

            y_pred = mlp.predict(X_test)
            acc_test = accuracy_score(y_test, y_pred)
            acc_scores['Num Neurons'].append(num_neurons)
            acc_scores['Accuracy'].append(acc_test)

    return acc_scores

# Passo 2: Escolher a melhor função de ativação e variar o número de neurônios
activation_functions = ['identity', 'logistic', 'tanh', 'relu']
num_neurons_values = [2, 451, 900, 226, 675]

for base, X, y in [('df1', df1.iloc[:, :-1], df1.iloc[:, -1]),
                  ('df2', df2.iloc[:, :-1], df2.iloc[:, -1]),
                  ('df_vgg_avg', df_vgg_avg.iloc[:, :-1], df_vgg_avg.iloc[:, -1]),
                  ('df_vgg_max', df_vgg_max.iloc[:, :-1], df_vgg_max.iloc[:, -1]),
                  ('dataset1_pca', dataset1_pca.iloc[:, :-1], dataset1_pca.iloc[:, -1]),
                  ('dataset2_pca', dataset2_pca.iloc[:, :-1], dataset2_pca.iloc[:, -1])]:
    plt.figure(figsize=(12, 6))
    for activation in activation_functions:
        acc_scores = experiment_mlp_neurons(X, y, num_neurons_values, activation, 1000, 0.001)
        plt.plot(acc_scores['Num Neurons'], acc_scores['Accuracy'], label=f'{activation}')

    plt.title(f'Experimento 2 - Base: {base}')
    plt.xlabel('Número de Neurônios')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.show()


In [ ]:
# Função para realizar experimentos com MLP (Passo 3)
def experiment_mlp_iterations(X, y, num_neurons, activation, max_iter_list, learning_rate):
    acc_scores = {'Num Iterations': [], 'Accuracy': []}

    for max_iter in max_iter_list:
        for test_size in [0.3, 0.2, 0.1]:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

            mlp = MLPClassifier(hidden_layer_sizes=num_neurons, activation=activation, max_iter=max_iter,
                                learning_rate_init=learning_rate, random_state=42)

            mlp.fit(X_train, y_train)

            y_pred = mlp.predict(X_test)
            acc_test = accuracy_score(y_test, y_pred)
            acc_scores['Num Iterations'].append(max_iter)
            acc_scores['Accuracy'].append(acc_test)

    return acc_scores

# Passo 3: Escolher o melhor número de neurônios e variar o número de iterações
best_num_neurons = 451  # Substitua pelo melhor valor encontrado no Passo 2
max_iterations_values = [100, 1000, 5000]

for base, X, y in [('df1', df1.iloc[:, :-1], df1.iloc[:, -1]),
                  ('df2', df2.iloc[:, :-1], df2.iloc[:, -1]),
                  ('df_vgg_avg', df_vgg_avg.iloc[:, :-1], df_vgg_avg.iloc[:, -1]),
                  ('df_vgg_max', df_vgg_max.iloc[:, :-1], df_vgg_max.iloc[:, -1]),
                  ('dataset1_pca', dataset1_pca.iloc[:, :-1], dataset1_pca.iloc[:, -1]),
                  ('dataset2_pca', dataset2_pca.iloc[:, :-1], dataset2_pca.iloc[:, -1])]:
    plt.figure(figsize=(12, 6))
    acc_scores = experiment_mlp_iterations(X, y, best_num_neurons, 'relu', max_iterations_values, 0.001)
    plt.plot(acc_scores['Num Iterations'], acc_scores['Accuracy'], label='relu')

    plt.title(f'Experimento 3 - Base: {base}')
    plt.xlabel('Número de Iterações')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.show()



In [ ]:
# Função para realizar experimentos com MLP (Passo 4)
def experiment_mlp_learning_rate(X, y, num_neurons, activation, max_iter, learning_rate_list):
    acc_scores = {'Learning Rate': [], 'Accuracy': []}

    for learning_rate in learning_rate_list:
        for test_size in [0.3, 0.2, 0.1]:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

            mlp = MLPClassifier(hidden_layer_sizes=num_neurons, activation=activation, max_iter=max_iter,
                                learning_rate_init=learning_rate, random_state=42)

            mlp.fit(X_train, y_train)

            y_pred = mlp.predict(X_test)
            acc_test = accuracy_score(y_test, y_pred)
            acc_scores['Learning Rate'].append(learning_rate)
            acc_scores['Accuracy'].append(acc_test)

    return acc_scores

# Passo 4: Escolher o melhor número de iterações e variar a taxa de aprendizado
best_num_iterations = 1000  # Substitua pelo melhor valor encontrado no Passo 3
learning_rate_values = [0.001, 0.01, 0.1]

for base, X, y in [('df1', df1.iloc[:, :-1], df1.iloc[:, -1]),
                  ('df2', df2.iloc[:, :-1], df2.iloc[:, -1]),
                  ('df_vgg_avg', df_vgg_avg.iloc[:, :-1], df_vgg_avg.iloc[:, -1]),
                  ('df_vgg_max', df_vgg_max.iloc[:, :-1], df_vgg_max.iloc[:, -1]),
                  ('dataset1_pca', dataset1_pca.iloc[:, :-1], dataset1_pca.iloc[:, -1]),
                  ('dataset2_pca', dataset2_p)]

In [ ]:
# Teste de Friedman para Gaussian NB
# Cada linha é um dataset e cada coluna é um método (proporção de treinamento/teste)
accs = [
    [0.69, 0.70, 0.72, 0.78],  # HOG(16x16) 1764 Att
    [0.69, 0.73, 0.72, 0.78],  # HOG(20x20) 900 Att
    [0.65, 0.69, 0.67, 0.70],  # PCA HOG(16x16)
    [0.99, 0.98, 0.97, 0.99],  # VGG 16 (Pooling max)
    [0.99, 0.98, 0.97, 0.99]   # VGG 16 (Pooling avg)
]

stat, p = friedmanchisquare(*accs)

print('Estatística de Friedman: %.3f' % stat)
print('p-valor: %.3f' % p)

Estatística de Friedman: 15.836
p-valor: 0.003


In [ ]:

accs_np = np.array([
    [0.69, 0.70, 0.72, 0.78],  # HOG(16x16) 1764 Att
    [0.69, 0.73, 0.72, 0.78],  # HOG(20x20) 900 Att
    [0.65, 0.69, 0.67, 0.70],  # PCA HOG(16x16)
    [0.99, 0.98, 0.97, 0.99],  # VGG 16 (Pooling max)
    [0.99, 0.98, 0.97, 0.99]   # VGG 16 (Pooling avg)
])

# teste de Nemenyi
result = sp.posthoc_nemenyi_friedman(accs_np.T)

print(result)

          0         1         2         3         4
0  1.000000  0.900000  0.707384  0.316855  0.316855
1  0.900000  1.000000  0.580482  0.449813  0.449813
2  0.707384  0.580482  1.000000  0.015044  0.015044
3  0.316855  0.449813  0.015044  1.000000  0.900000
4  0.316855  0.449813  0.015044  0.900000  1.000000


In [ ]:
data = [
    [0.67, 0.72, 0.68, 0.65, 0.65, 0.65, 0.62, 0.63],  # HOG(16x16) 1764 Att 10-fold CV
    [0.67, 0.68, 0.61, 0.64, 0.64, 0.63, 0.64, 0.64],  # HOG(16x16) 1764 Att 70/30
    [0.63, 0.60, 0.65, 0.63, 0.60, 0.61, 0.65, 0.65],  # HOG(16x16) 1764 Att 80/20
    [0.65, 0.65, 0.66, 0.65, 0.67, 0.65, 0.63, 0.64],  # HOG(16x16) 1764 Att 90/10
    [0.63, 0.66, 0.59, 0.59, 0.61, 0.56, 0.59, 0.59],  # HOG(20x20) 900 Att 10-fold CV
    [0.60, 0.57, 0.63, 0.64, 0.61, 0.63, 0.60, 0.61],  # HOG(20x20) 900 Att 70/30
    [0.69, 0.60, 0.65, 0.55, 0.61, 0.59, 0.53, 0.61],  # HOG(20x20) 900 Att 80/20
    [0.67, 0.66, 0.64, 0.65, 0.65, 0.67, 0.67, 0.64],  # HOG(20x20) 900 Att 90/10
    [0.65, 0.63, 0.64, 0.60, 0.60, 0.59, 0.58, 0.59],  # PCA HOG(16x16) 10-fold CV
    [0.70, 0.69, 0.69, 0.67, 0.66, 0.62, 0.65, 0.67],  # PCA HOG(16x16) 70/30
    [0.65, 0.65, 0.64, 0.65, 0.70, 0.66, 0.68, 0.63],  # PCA HOG(16x16) 80/20
    [0.64, 0.63, 0.63, 0.61, 0.62, 0.63, 0.60, 0.61],  # PCA HOG(16x16) 90/10
    [0.96, 0.95, 0.96, 0.96, 0.96, 0.96, 0.96, 0.95],  # VGG 16 (Pooling max) 10-fold CV
    [0.95, 0.96, 0.96, 0.96, 0.96, 0.95, 0.96, 0.96],  # VGG 16 (Pooling max) 70/30
    [0.95, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96],  # VGG 16 (Pooling max) 80/20
    [0.97, 0.97, 0.96, 0.96, 0.96, 0.96, 0.97, 0.96],  # VGG 16 (Pooling max) 90/10
    [0.97, 0.95, 0.96, 0.96, 0.96, 0.96, 0.96, 0.95],  # VGG 16 (Pooling avg) 10-fold CV
    [0.96, 0.95, 0.96, 0.96, 0.94, 0.94, 0.96, 0.94],  # VGG 16 (Pooling avg) 70/30
    [0.95, 0.96, 0.96, 0.96, 0.95, 0.95, 0.96, 0.96],  # VGG 16 (Pooling avg) 80/20
    [0.97, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96]   # VGG 16 (Pooling avg) 90/10
]

# Realize o teste de Friedman
stat, p = stats.friedmanchisquare(*data)

print('Estatística de Friedman: %.3f' % stat)
print('p-valor: %.3f' % p)

Estatística de Friedman: 133.591
p-valor: 0.000


In [ ]:
accs_np = np.array([
    [0.67, 0.72, 0.68, 0.65, 0.65, 0.65, 0.62, 0.63],  # HOG(16x16) 1764 Att 10-fold CV
    [0.67, 0.68, 0.61, 0.64, 0.64, 0.63, 0.64, 0.64],  # HOG(16x16) 1764 Att 70/30
    [0.63, 0.60, 0.65, 0.63, 0.60, 0.61, 0.65, 0.65],  # HOG(16x16) 1764 Att 80/20
    [0.65, 0.65, 0.66, 0.65, 0.67, 0.65, 0.63, 0.64],  # HOG(16x16) 1764 Att 90/10
    [0.63, 0.66, 0.59, 0.59, 0.61, 0.56, 0.59, 0.59],  # HOG(20x20) 900 Att 10-fold CV
    [0.60, 0.57, 0.63, 0.64, 0.61, 0.63, 0.60, 0.61],  # HOG(20x20) 900 Att 70/30
    [0.69, 0.60, 0.65, 0.55, 0.61, 0.59, 0.53, 0.61],  # HOG(20x20) 900 Att 80/20
    [0.67, 0.66, 0.64, 0.65, 0.65, 0.67, 0.67, 0.64],  # HOG(20x20) 900 Att 90/10
    [0.65, 0.63, 0.64, 0.60, 0.60, 0.59, 0.58, 0.59],  # PCA HOG(16x16) 10-fold CV
    [0.70, 0.69, 0.69, 0.67, 0.66, 0.62, 0.65, 0.67],  # PCA HOG(16x16) 70/30
    [0.65, 0.65, 0.64, 0.65, 0.70, 0.66, 0.68, 0.63],  # PCA HOG(16x16) 80/20
    [0.64, 0.63, 0.63, 0.61, 0.62, 0.63, 0.60, 0.61],  # PCA HOG(16x16) 90/10
    [0.96, 0.95, 0.96, 0.96, 0.96, 0.96, 0.96, 0.95],  # VGG 16 (Pooling max) 10-fold CV
    [0.95, 0.96, 0.96, 0.96, 0.96, 0.95, 0.96, 0.96],  # VGG 16 (Pooling max) 70/30
    [0.95, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96],  # VGG 16 (Pooling max) 80/20
    [0.97, 0.97, 0.96, 0.96, 0.96, 0.96, 0.97, 0.96],  # VGG 16 (Pooling max) 90/10
    [0.97, 0.95, 0.96, 0.96, 0.96, 0.96, 0.96, 0.95],  # VGG 16 (Pooling avg) 10-fold CV
    [0.96, 0.95, 0.96, 0.96, 0.94, 0.94, 0.96, 0.94],  # VGG 16 (Pooling avg) 70/30
    [0.95, 0.96, 0.96, 0.96, 0.95, 0.95, 0.96, 0.96],  # VGG 16 (Pooling avg) 80/20
    [0.97, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96]   # VGG 16 (Pooling avg) 90/10
])

# Realize o teste de Nemenyi
result = sp.posthoc_nemenyi_friedman(accs_np.T)

print(result)

          0         1         2         3         4         5         6   \
0   1.000000  0.900000  0.900000  0.900000  0.867949  0.900000  0.900000   
1   0.900000  1.000000  0.900000  0.900000  0.900000  0.900000  0.900000   
2   0.900000  0.900000  1.000000  0.900000  0.900000  0.900000  0.900000   
3   0.900000  0.900000  0.900000  1.000000  0.900000  0.900000  0.900000   
4   0.867949  0.900000  0.900000  0.900000  1.000000  0.900000  0.900000   
5   0.900000  0.900000  0.900000  0.900000  0.900000  1.000000  0.900000   
6   0.900000  0.900000  0.900000  0.900000  0.900000  0.900000  1.000000   
7   0.900000  0.900000  0.900000  0.900000  0.826110  0.900000  0.900000   
8   0.900000  0.900000  0.900000  0.900000  0.900000  0.900000  0.900000   
9   0.900000  0.900000  0.900000  0.900000  0.547169  0.770322  0.854003   
10  0.900000  0.900000  0.900000  0.900000  0.900000  0.900000  0.900000   
11  0.900000  0.900000  0.900000  0.900000  0.900000  0.900000  0.900000   
12  0.616901

In [ ]:

# Seus dados
data = [
    [0.67, 0.57, 0.61, 0.58, 0.60, 0.57, 0.59, 0.57, 0.58, 0.57],  # Dataset 1
    [0.66, 0.61, 0.68, 0.54, 0.63, 0.58, 0.54, 0.46, 0.64, 0.56],  # Dataset 2
    [0.62, 0.60, 0.56, 0.52, 0.53, 0.51, 0.61, 0.56, 0.65, 0.57],  # Dataset 3
    [0.68, 0.59, 0.61, 0.52, 0.52, 0.54, 0.59, 0.54, 0.59, 0.58],  # Dataset 4
    [0.69, 0.61, 0.66, 0.60, 0.64, 0.60, 0.61, 0.59, 0.63, 0.60],  # Dataset 5
    [0.60, 0.71, 0.63, 0.59, 0.61, 0.58, 0.68, 0.63, 0.59, 0.51],  # Dataset 6
    [0.72, 0.64, 0.64, 0.63, 0.62, 0.59, 0.59, 0.61, 0.60, 0.54],  # Dataset 7
    [0.70, 0.61, 0.69, 0.53, 0.63, 0.63, 0.59, 0.59, 0.62, 0.58],  # Dataset 8
    [0.68, 0.59, 0.64, 0.59, 0.62, 0.59, 0.61, 0.60, 0.60, 0.58],  # Dataset 9
    [0.65, 0.60, 0.67, 0.60, 0.61, 0.51, 0.58, 0.48, 0.59, 0.58],  # Dataset 10
    [0.67, 0.59, 0.54, 0.64, 0.67, 0.54, 0.63, 0.56, 0.63, 0.58],  # Dataset 11
    [0.62, 0.61, 0.59, 0.59, 0.58, 0.59, 0.56, 0.63, 0.63, 0.63],  # Dataset 12
    [0.98, 0.99, 0.98, 0.98, 0.98, 0.99, 0.98, 0.99, 0.98, 0.98],  # Dataset 13
    [0.98, 0.98, 0.95, 1.00, 0.98, 0.96, 0.99, 0.99, 0.99, 1.00],  # Dataset 14
    [1.00, 0.99, 0.98, 0.97, 0.98, 0.98, 0.97, 0.97, 1.00, 0.89],  # Dataset 15
    [1.00, 0.99, 0.99, 0.98, 0.97, 0.98, 0.98, 0.98, 0.99, 0.98],  # Dataset 16
    [0.98, 0.99, 0.98, 0.98, 0.98, 0.99, 0.98, 1.00, 0.98, 0.98],  # Dataset 17
    [0.98, 0.96, 0.98, 0.99, 1.00, 0.99, 0.99, 0.99, 1.00, 1.00],  # Dataset 18
    [0.98, 0.99, 0.97, 0.98, 0.98, 0.98, 0.97, 0.97, 0.99, 0.97],  # Dataset 19
    [0.99, 0.99, 0.97, 0.99, 0.99, 1.00, 0.98, 0.97, 1.00, 0.97]   # Dataset 20
]

# Realize o teste de Friedman
stat, p = stats.friedmanchisquare(*data)

print('Estatística de Friedman: %.3f' % stat)
print('p-valor: %.50f' % p)

# Interpretação
alpha = 0.05
if p > alpha:
    print('Mesmas distribuições (falha em rejeitar H0)')
else:
    print('Distribuições diferentes (rejeita H0)')



Estatística de Friedman: 150.354
p-valor: 0.00000000000000000000018729093976745308544713219836
Distribuições diferentes (rejeita H0)


In [ ]:
import numpy as np

# Definir a matriz A e o vetor b
A = np.array([-6,-2,1, 2, 3, 5, 7, 10, 11,20,36])
b = np.array([36,4,1, 4, 9, 25, 49, 100, 121, 400, 1296])

# Calcular a transposta de A
A_T = A.transpose()

# Calcular A^T A
AtA = np.matmul(A_T, A)

# Calcular a inversa de A^T A
inv_AtA = np.linalg.inv(AtA)

# Calcular A^T b
ATb = np.matmul(A_T, b)

# Calcular β
beta = np.matmul(inv_AtA, ATb)

# Imprimir o resultado
print(beta)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional